The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [ ]:
!pip install d2l==0.17.1


# Implementação Concisa de Regressão Linear
:label:`sec_linear_concise`


Amplo e intenso interesse em *deep learning* nos últimos anos
inspiraram empresas, acadêmicos e amadores
para desenvolver uma variedade de estruturas de código aberto maduras
para automatizar o trabalho repetitivo de implementação
algoritmos de aprendizagem baseados em gradiente.
Em :numref:`sec_linear_scratch`, contamos apenas com
(i) tensores para armazenamento de dados e álgebra linear;
e (ii) auto diferenciação para cálculo de gradientes.
Na prática, porque iteradores de dados, funções de perda, otimizadores,
e camadas de rede neural
são tão comuns que as bibliotecas modernas também implementam esses componentes para nós.

Nesta seção, (**mostraremos como implementar
o modelo de regressão linear**) de:numref:`sec_linear_scratch`
(**de forma concisa, usando APIs de alto nível**) de estruturas de *deep learning*.


## Gerando the Dataset

Para começar, vamos gerar o mesmo conjunto de dados como em
:numref:`sec_linear_scratch`.


In [1]:
import numpy as np
import tensorflow as tf
from d2l import tensorflow as d2l

In [2]:
true_w = tf.constant([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

## Lendo o Dataset

Em vez de usar nosso próprio iterador,
podemos [**chamar a API existente em uma estrutura para ler os dados.**]
Passamos *`features`* e *`labels`* como argumentos e especificamos *`batch_size`*
ao instanciar um objeto iterador de dados.
Além disso, o valor booleano `is_train`
indica se ou não
queremos que o objeto iterador de dados embaralhe os dados
em cada época (passe pelo conjunto de dados).


In [3]:
def load_array(data_arrays, batch_size, is_train=True):  #@save
    """Construct a TensorFlow data iterator."""
    dataset = tf.data.Dataset.from_tensor_slices(data_arrays)
    if is_train:
        dataset = dataset.shuffle(buffer_size=1000)
    dataset = dataset.batch(batch_size)
    return dataset

In [4]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

Now we can use `data_iter` in much the same way as we called
the `data_iter` function in :numref:`sec_linear_scratch`.
To verify that it is working, we can read and print
the first minibatch of examples.
Comparing with :numref:`sec_linear_scratch`,
here we use `iter` to construct a Python iterator and use `next` to obtain the first item from the iterator.


In [5]:
next(iter(data_iter))

(<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
 array([[-0.4333908 ,  0.668291  ],
        [-0.12056218,  1.6938822 ],
        [-0.61093557,  0.66606677],
        [-0.42496178,  0.64752823],
        [-0.9900723 , -0.08506326],
        [ 0.5402116 ,  0.11740936],
        [-0.09974687, -1.0370846 ],
        [-0.8218537 ,  0.3895554 ],
        [-0.40174934,  0.69312775],
        [ 0.20610306,  1.057085  ]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[ 1.0540864],
        [-1.8192354],
        [ 0.7101214],
        [ 1.1270143],
        [ 2.5089824],
        [ 4.8952   ],
        [ 7.524812 ],
        [ 1.2295262],
        [ 1.0272019],
        [ 1.023846 ]], dtype=float32)>)

## Definindo o Modelo


Quando implementamos a regressão linear do zero
em :numref:`sec_linear_scratch`,
definimos nossos parâmetros de modelo explicitamente
e codificamos os cálculos para produzir saída
usando operações básicas de álgebra linear.
Você *deveria* saber como fazer isso.
Mas quando seus modelos ficam mais complexos,
e uma vez que você tem que fazer isso quase todos os dias,
você ficará feliz com a ajuda.
A situação é semelhante a codificar seu próprio blog do zero.
Fazer uma ou duas vezes é gratificante e instrutivo,
mas você seria um péssimo desenvolvedor da web
se toda vez que você precisava de um blog você passava um mês
reinventando tudo.

Para operações padrão, podemos [**usar as camadas predefinidas de uma estrutura,**]
o que nos permite focar especialmente
nas camadas usadas para construir o modelo
em vez de ter que se concentrar na implementação.
Vamos primeiro definir uma variável de modelo `net`,
que se refere a uma instância da classe `Sequential`.
A classe `Sequential` define um contêiner
para várias camadas que serão encadeadas.
Dados dados de entrada, uma instância `Sequential` passa por
a primeira camada, por sua vez passando a saída
como entrada da segunda camada e assim por diante.
No exemplo a seguir, nosso modelo consiste em apenas uma camada,
portanto, não precisamos realmente de `Sequencial`.
Mas como quase todos os nossos modelos futuros
envolverão várias camadas,
vamos usá-lo de qualquer maneira apenas para familiarizá-lo
com o fluxo de trabalho mais padrão.

Lembre-se da arquitetura de uma rede de camada única, conforme mostrado em :numref:`fig_single_neuron`.
Diz-se que a camada está *totalmente conectada*
porque cada uma de suas entradas está conectada a cada uma de suas saídas
por meio de uma multiplicação de matriz-vetor.


: begin_tab: `pytorch`
No PyTorch, a camada totalmente conectada é definida na classe `Linear`. Observe que passamos dois argumentos para `nn.Linear`. O primeiro especifica a dimensão do recurso de entrada, que é 2, e o segundo é a dimensão do recurso de saída, que é um escalar único e, portanto, 1.


No Keras, a camada totalmente conectada é definida na classe `Dense`. Como queremos gerar apenas uma única saída escalar, definimos esse número como 1.

É importante notar que, por conveniência,
Keras não exige que especifiquemos
a forma de entrada para cada camada.
Então, aqui, não precisamos dizer a Keras
quantas entradas vão para esta camada linear.
Quando tentamos primeiro passar dados por meio de nosso modelo,
por exemplo, quando executamos `net (X)` mais tarde,
Keras inferirá automaticamente o número de entradas para cada camada.
Descreveremos como isso funciona com mais detalhes posteriormente.


In [6]:
# `keras` is the high-level API for TensorFlow
net = tf.keras.Sequential()
net.add(tf.keras.layers.Dense(1))

## Inicializando os Parâmetros do Modelo


Antes de usar `net`, precisamos (**inicializar os parâmetros do modelo,**)
como os pesos e *bias* no modelo de regressão linear.
As estruturas de *deep learning* geralmente têm uma maneira predefinida de inicializar os parâmetros.
Aqui especificamos que cada parâmetro de peso
deve ser amostrado aleatoriamente a partir de uma distribuição normal
com média 0 e desvio padrão 0,01.
O parâmetro bias será inicializado em zero.


O módulo *`initializers`* no TensorFlow fornece vários métodos para a inicialização dos parâmetros do modelo. A maneira mais fácil de especificar o método de inicialização no Keras é ao criar a camada especificando *`kernel_initializer`*. Aqui, recriamos o `net` novamente.


In [7]:
initializer = tf.initializers.RandomNormal(stddev=0.01)
net = tf.keras.Sequential()
net.add(tf.keras.layers.Dense(1, kernel_initializer=initializer))

O código acima pode parecer simples, mas você deve observar
que algo estranho está acontecendo aqui.
Estamos inicializando parâmetros para uma rede
mesmo que o Keras ainda não saiba
quantas dimensões a entrada terá!
Pode ser 2 como em nosso exemplo ou pode ser 2.000.
O Keras nos permite fugir do problema com isso porque, nos bastidores,
a inicialização é, na verdade, *adiada*.
A inicialização real ocorrerá apenas
quando tentamos, pela primeira vez, passar dados pela rede.
Apenas tome cuidado para lembrar que, uma vez que os parâmetros
ainda não foram inicializados,
não podemos acessá-los ou manipulá-los.


## Definindo a Função de Perda


A classe `MeanSquaredError` calcula o erro quadrático médio, também conhecido como norma $L_2$ quadrada.
Por padrão, ela retorna a perda média sobre os exemplos.


In [8]:
loss = tf.keras.losses.MeanSquaredError()

## Definindo o Algoritmo de Otimização


O gradiente descendente estocástico de *minibatch* é uma ferramenta padrão
para otimizar redes neurais
e, portanto, Keras oferece suporte ao lado de uma série de
variações deste algoritmo no módulo `otimizadores`.
O gradiente descendente estocástico de *minibatch* requer apenas que
definamos o valor `learning_rate`, que é definido como 0,03 aqui.


In [9]:
trainer = tf.keras.optimizers.SGD(learning_rate=0.03)

## Treinamento


Você deve ter notado que expressar nosso modelo por meio
APIs de alto nível de uma estrutura de *deep learning*
requer comparativamente poucas linhas de código.
Não tivemos que alocar parâmetros individualmente,
definir nossa função de perda ou implementar o gradiente descendente estocástico de *minibatch*.
Assim que começarmos a trabalhar com modelos muito mais complexos,
as vantagens das APIs de alto nível aumentarão consideravelmente.
No entanto, uma vez que temos todas as peças básicas no lugar,
[**o loop de treinamento em si é surpreendentemente semelhante
ao que fizemos ao implementar tudo do zero.**]

Para refrescar sua memória: para anguns números de épocas,
faremos uma passagem completa sobre o conjunto de dados (*`train_data`*),
pegando iterativamente um *minibatch* de entradas
e os *labels* de verdade fundamental correspondentes.
Para cada *minibatch*, passamos pelo seguinte ritual:

* Gerar previsões chamando `net (X)` e calcular a perda `l` (a propagação direta).
* Calcular gradientes executando a retropropagação.
* Atualizar os parâmetros do modelo invocando nosso otimizador.

Para uma boa medida, calculamos a perda após cada época e a imprimimos para monitorar o progresso.


In [10]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        with tf.GradientTape() as tape:
            l = loss(net(X, training=True), y)
        grads = tape.gradient(l, net.trainable_variables)
        trainer.apply_gradients(zip(grads, net.trainable_variables))
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000263


epoch 2, loss 0.000089


epoch 3, loss 0.000089


Abaixo, nós [**comparamos os parâmetros do modelo aprendidos pelo treinamento em dados finitos
e os parâmetros reais**] que geraram nosso *dataset*.
Para acessar os parâmetros,
primeiro acessamos a camada que precisamos de `net`
e, em seguida, acessamos os pesos e a polarização dessa camada.
Como em nossa implementação do zero,
observe que nossos parâmetros estimados são
perto de suas contrapartes verdadeiras.


In [11]:
w = net.get_weights()[0]
print('error in estimating w', true_w - tf.reshape(w, true_w.shape))
b = net.get_weights()[1]
print('error in estimating b', true_b - b)

error in estimating w tf.Tensor([0.00035477 0.00064921], shape=(2,), dtype=float32)
error in estimating b [-0.00018072]


## Resumo


* Usando as APIs de alto nível do TensorFlow, podemos implementar modelos de maneira muito mais concisa.
* No TensorFlow, o módulo `data` fornece ferramentas para processamento de dados, o módulo` keras` define um grande número de camadas de rede neural e funções de perda comuns.
* O módulo *`initializers`* do TensorFlow fornece vários métodos para a inicialização dos parâmetros do modelo.
* A dimensionalidade e o armazenamento são inferidos automaticamente (mas tome cuidado para não tentar acessar os parâmetros antes de serem inicializados).


## Exercícios


1. Revise a documentação do TensorFlow para ver quais funções de perda e métodos de inicialização são fornecidos. Substitua a perda pela perda de Huber.

[Discussions](https://discuss.d2l.ai/t/204)


<!--stackedit_data:
eyJoaXN0b3J5IjpbLTUxMjc1MTc4MiwxMzgxNzE4MjMxLDE1MT
Y2Nzk5ODgsLTMwMDU2OTMzNSwtNTUxNzY3NTUsMzYzNjY2MzMs
LTY1Mjk5MTk1OCwtMjE0NTk5NTMwN119
-->
